In [8]:
# Hydrolight_MFile_reader_py
# Jesse Bausell
# September 25, 2019
#
# This python notebook reformats a series of Hydrolight-generated m-files (radiative transfer 
# outputs) into hdf5 files. This enables easier access to data for investigators, who can work
# with structured variables inside the hdf5 files rather than unweildy ascii files, which are 
# difficult to utilize on a large scale. See GitHub readme for more details.

In [9]:
### 1. Import libraries necessary to run the program 
import numpy as np # numpy
import h5py # hdf5 library
import re # regular expression library
import os # operating system library
import csv # comma-separated variables reader/writer library
from tkinter import filedialog as fd # Filedialog library


In [10]:
### 2. Compile data from m-file into a python dictionary

def ascii_MFILE_compiler(mfile_pthwy):
    """Compiles a m-file into a multi-layerd, nested python dictionary. For each parameters
    Dictionary keys take on the name of the m-file column headers. If a variable changes by depth
    as well as wavelength, these parameters are nested within the dictionary element. Actual 
    variable matrix is denoted as "Values" (generally displayed as wavelength x depth)    
    Input: mfile_pthwy - m-file pathway
    Output: HE53 - python dictionary"""
    with open(mfile_pthwy) as raw: # open m-file
        HE53 = {} # Create an empty dictionary
        HE53['Header'] = [] # Create an empty list for header lines 1-4 (0-3)
        keY = 0 # key to switch between AOPs (keY = 0) and IOPs (keY = 1)
        # sections have two different header formats and thus require processing differences
        for i in np.arange(4):
            # For-loop removes the first three header lines
            HE53['Header'].append(raw.readline()) # Add header line to list to be reformatted later
        while 1:
            # Cycle through m-file line by line continuously.
            raWtitlE = raw.readline()[:-1] # Take ONE file line (this will always be the first header)
            if raWtitlE == '':
                # If the end of the m-file has been reached...
                break # Break while-loop
            else:
                # If the end of the m-file has been reached. Entertain two different options
                if 'component' in raWtitlE: 
                    # IOPs (absorption, scatter, backscatter, and backscatter-scatter fractions)
                    # are reported as total values, and the "components" that are attributed to 
                    # pure-water, phytoplankton, CDOM, and minerals. This if-statement deals with
                    # components and their unique header format.
                    titlE = raWtitlE.split('"') # Get the second header line of component section
                    namE = titlE[3] # Isolate "Component #" This will later become a dictionary element 
                else:
                    # If the m-file section is not an IOP "Component", but rather "total" IOPs or AOPs
                    if keY == 0:
                        # If m-file section comprises AOPs (key = 0)
                        # Split first header title into "-separated sections (line below)
                        titlE = raWtitlE.split('"') 
                        raw.readline() # skip second header title (blank and useless)
                        namE = titlE[1] # locate varible name  first title header. Assign variable
                        uniT = titlE[3] # locate units
                        diM = np.asarray(titlE[4].split(),dtype=int) # Find x & y dimensions of AOP 
                        # matrix. These will be used later on to read in the data.
                    else:
                        # If the m-file section comprises "Total IOPs"
                        titlE = linE.split() # split the first header line by empty space
                        namE = titlE[0] # locate varible name first title header. Assign variable
                        units = '1/m' # Assign units. All the same for IOPs (except bb/b fraction)
                        diM = np.asarray([titlE[-2],titlE[-1]],dtype=int) # Find x & y dimensions of AOP 
                        # matrix. These will be used later on to read in the data.
                # Select third header line. This contains data columns for m-file section (see below)
                # first, eliminate quotation marks replace "in air" with -1 to quantify depth above water
                fielDS = raw.readline()[:-1].replace('"',' ').replace("in air","-1") # see annotation above
                par_IND = fielDS.find('(') # cut out all parentheses in third header line
                fielDS = fielDS[:par_IND].split() # split third header line by empty spaces to generate column headers
                HE53[namE] = {} # Create an empty dictionary with name of top header. Nest it in original dictionary
                HE53[namE]['units'] = uniT # Create "units" element within nested dictionary
                DATA = np.zeros(diM)*np.nan # Create nan matrix with dimensions of m-file data
                for i in np.arange(diM[0]):
                    # for-loop cycles through each line of data, converts it from a string into a
                    # numpy array, and adds it onto DATA (nan matrix).
                    # First, take a line (linE) of numerical data. Strip away quotations, "in air",
                    # and end-of-line character (\n)
                    linE = raw.readline().replace('"',' ').replace("in air","-1").replace('\n','')
                    try:
                        # Attempt to convert string into a numpy array and insert it into DATA
                        DATA[i,:] = np.asarray(linE.split(),dtype=np.float32)
                    except:
                        # If an error message is generated, it means that for-loop has accidentally
                        # reached the next header. This is prone to happen in the IOP data sections 
                        # of the m-file because the number of rows is OVER REPORTED. linE is saved and 
                        # converted into the first header of the next section of data.
                        DATA = DATA[:i,:] # Index the remaining nan rows out of the data matrix
                        keY=+1 # Increase switch to one in order to start reading in IOP data
                        break # BREAK FOR-LOOP!
                HE53[namE][fielDS[0]] = DATA[:,0] # Create first dictionary element within newly created nested for-loop. 
                fielDS.pop(0) # Remove the name of the first dictionary element from column headers
                DATA = DATA[:,1:] # Remove the first column from the data matrix (corresponding to dictionary element)
                try:
                    # Determine whether the column headers are actually water column depths. 
                    HE53[namE]['Depth'] = np.asarray(fielDS,dtype=np.float32) # If fields are depths, assign them to a dictionary element
                    HE53[namE]['Values'] = DATA # Assign data matrix to a field called "Values" (wavelength x depth)
                except:
                    # If the column headers are NOT depths, and are comprised of words....
                    for i, j in enumerate(fielDS):
                        # Make each column header into an element name in the nested dictionary
                        HE53[namE][j] = DATA[:,i] # Match the column header with corresponding matrix column 
        return(HE53) # Return the dictionary

In [11]:
### 3. Compile data from m-file into a python dictionary

def hdf5_fileWRITER(filE_NAME,HE53_dict):
    """Takes the python dictionary that was generated by ascii_MFILE_compiler and writes them
    into a hdf5 (.h5) file. Data within hdf5 file is formatted the same as the aforementioned 
    python dictionary. User should note however that "bb/b ratio" will be changed to "bb fraction"
    in all headers.
    Inputs: 
        filE_NAME - name of future hdf5 file that will contain dictionary data
        HE53_dict - dictionary formatted by ascii_MFILE_compiler
    Outputs:
        filE_NAME.h5 - hdf5 file containing python dictionary data"""
    with h5py.File(filE_NAME + '.h5','w') as hf: # Create an open hdf5 file for writing.
        for k in HE53_dict:
            # for-loop disects the m-file dictionary and writes data and dictionary elements
            # into a hdf5 file.
            if k != 'Header':
                # For all dictionary elements EXCEPT for "Header", which contains the first four
                # lines of the m-file...
                if '/' in k or '(' in k:
                    # If a header denotes a m-file IOP component...
                    k_nuggets = k.split() # split string by white spaces 
                    if k_nuggets[0] == 'bb/b': 
                        # if dictionary key contains "bb/b"...
                        k_nuggets[0] = 'bb fraction' # change "bb/b" to "bb fraction"   
                    name = k_nuggets[0] + ' ' + k_nuggets[-2] + k_nuggets[-1] # add "component #" 
                else:
                    # If a header does not denote a m-file IOP component 
                    # (e.g. AOP or total IOP variable)
                    name = k # Use the original name of the dictionary key
                # create a "python dictionary" in recently-created hdf5 file...
                hf.create_group(name) # Create a key/element in hdf5 file based on nested python dictionary
                for l in HE53_dict[k]:
                    # Within the python dictionary, take all elements (keys and data) and incorporate them 
                    # into hdf5 file
                    hf[name][l] = HE53_dict[k][l] # Create new nested element with data in hdf5 file
            else:
                # When program encounters the Header element...
                hf['Meta Data'] = HE53_dict['Header'][0] # Insert first line of 4-line header into hdf5 file
                

In [12]:
### 4. Define a program that can create a new folder for storing data files on computer.

def createFolder(directory):
    """ createFolder searches for a dirctory specified by the user. If there is none, it creates one"""
    try:
        if not os.path.exists(directory): # If the folder doesn't exist
            os.makedirs(directory) # Create a folder
    except OSError: # If there is an error other than a non-existant folder
        print ('Error: Creating directory. ' +  directory) # report error and shut down createFolder


In [13]:
def selecT_HE53_vars(HE53,mfile_name):
    """Select distinct variables from python dictionary for publication in csv file.
    Inputs:
        HE53 - python dictionary with data extracted from m-file
        mfile_name - original name of m-file
    Outputs: 
        ITR - iteration, or series number of m-file/hdf5 file (LogNorm_Draw_Hydrolight creates numerically ordered files)
        Chl - input chlorophyll
        CDOM - input Gelbstoff absorption (440 nm)
        TSS - input total suspended sediment/total suspended matter
        kd320 - diffuse attenuation coefficient 320 nm
        kd780 - diffuse attenuation coefficient 780 nm
        Kd412 - diffuse attenuation coefficient 412 nm
        Kd667 - diffuse attenuation coefficient 667 nm
        aph443 - phytoplankton absorption 443 nm
        ag443 - Galbstoff absorption 443 nm
        amin443 - mineral absorption 443 nm
        bb_ph667 - phytoplankton backscattering 667 nm
        bb_min667 - mineral backscattering 667 nm
        bb_phFrac - phytoplankton backscatter/scatter fraction 
        bb_minFrac -mineral backscatter/scatter fraction"""
    # Determine relative iteration of the m-file using header     
    itr_IND = mfile_name.find('itr_')+4
    txt_IND = mfile_name.find('.txt')
    ITR = int(mfile_name[itr_IND:txt_IND])
    # Find Kd320 and Kd780
    kd320 = HE53['Kd']['Values'][0,0]
    kd780 = HE53['Kd']['Values'][-1,0]
    # Find Kd412 and Kd670
    Kd412 = HE53['Kd']['Values'][2,0]
    Kd667 = HE53['Kd']['Values'][-2,0]
    # Find CDOM, phytoplankton and mineral absorption at 443 nm
    aph443 = HE53['a (1/m) for component  2']['Values'][3,0]
    ag443 = HE53['a (1/m) for component  3']['Values'][3,0]
    amin443 = HE53['a (1/m) for component  4']['Values'][3,0]
    # Find phytoplankton and mineral backscattering at 667 nm
    bb_ph667 = HE53['bb (1/m) for component  2']['Values'][-2,0]
    bb_min667 = HE53['bb (1/m) for component  4']['Values'][-2,0]
    # Find Chl, CDOM (ag440) and TSS based on the first Header line in the m-file
    titlE = HE53["Header"][0].split(',') 
    Chl_IND = titlE[1].find('=')+1
    Chl = float(titlE[1][Chl_IND:])
    CDOM_IND = titlE[2].find('=')+1
    CDOM = float(titlE[2][CDOM_IND:])
    TSS_IND = titlE[3].find('=')+1
    TSS = float(titlE[3][TSS_IND:])    
    # Find bb_phFrac and bb_minFrac
    bb_phFrac = HE53['bb/b ratio for component  2']['Values'][0,0]
    bb_minFrac = HE53['bb/b ratio for component  4']['Values'][0,0]
    return(ITR,Chl,CDOM,TSS,kd320,kd780,Kd412,Kd667,aph443,ag443,amin443,bb_ph667,bb_min667,bb_phFrac,bb_minFrac)

In [14]:
### 6. This cell is where MFile_reader_py is executed from. The code 
## 6a. Create the necessary file pathways and variables to compile m-files
template_dir = fd.askdirectory() # Select directory containing m-files
if '/' in template_dir:
    # If files are on a mac
    dasH = '/' # Folder separator for directory pathway
else:
    # If files are on a pc
    dasH = '\\' # Folder separator for directory pathway
mfile_pthwy = template_dir+dasH # Add a "/" or "\\" onto the end of the folder
dash_IND = [w.start() for w in re.finditer(dasH, mfile_pthwy)] # locate file separators
# Create a string indicating patyway of hdf5 files (below)
hdf5_file_pthwy = template_dir[:dash_IND[-2]+1] + 'hdf5' + dasH # See line above
csv_file_pthwy = template_dir[:dash_IND[-2]+1] + dasH # csv file pathway
createFolder(hdf5_file_pthwy) # Create a pathway on the computer for hdf5 files
matLISt = os.listdir(mfile_pthwy) # list all files in m-file directory
csv_DATA_MATRIX = np.zeros([len(matLISt),15])*np.nan # Create a data matrix to place 
mFILE_LIST = [] # Create an empty list for m-files to keep track of their original order
# variables earmarked for csv file
## 6b. Convert m-files into hdf5 format and extract necessary data into numpy matrix
for i,mFILE in enumerate(matLISt):
    # This for-loop cyles through m-files in user-selected folder. Data in each m-file (ascii)
    # is re-formatted into a hdf5 file (.h5) which is placed into a folder named "hdf5" 
    # adjacent to the user-selected m-file folder.
    if mFILE[0] == 'M' and '.txt' in mFILE:
        # Make sure that the file is indeed an m-file (ascii)
        # Compile information from m-file into python dictionary (line below)
        HE53 = ascii_MFILE_compiler(mfile_pthwy + mFILE) # Compiles m-file data into dictionary
        # extract data from dictionary and place it into numpy matrix (below)
        csv_DATA_MATRIX[i,:] = np.asarray(selecT_HE53_vars(HE53,mFILE),dtype=float) 
        t_IND = mFILE.find('.txt') # Find the index ".txt" part of ascii file
        filE_NAME_hdf5 = mFILE[:t_IND] # Cut off ".txt" and leave only the root of m-file name
        # Format dictionary into hdf5 file using root of m-file name (below)
        mFILE_LIST.append(mFILE) # Add m-file name to the list of m-files
        hdf5_fileWRITER(hdf5_file_pthwy+filE_NAME_hdf5,HE53) # Create hdf5 file
## 6c. Write numpy matrix into csv file
# 6c1. Prepare the numpy matrix
nan_IND = ~np.isnan(csv_DATA_MATRIX[:,0]) # Find rows that DO NOT contain nan values
csv_DATA_MATRIX = csv_DATA_MATRIX[nan_IND,:] # Exclude all matrix rows with nan values
# Determine bb/b fractions for phytoplankton and minerals (last two columsn of csv_DATA_MATRIX)
bb_ph = str(np.round(csv_DATA_MATRIX[0,-2],decimals=3)) # bb/b phytoplankton (string)
bb_min = str(np.round(csv_DATA_MATRIX[0,-1],decimals=3))# bb/b minerals (string)
csv_DATA_MATRIX = csv_DATA_MATRIX[:,:-2] # Remove the last two columns from the data matrix
csv_DATA_MATRIX[:,0] = csv_DATA_MATRIX[:,0] - min(csv_DATA_MATRIX[:,0]) #shift indices to 0-999
# 6c2. Write csv file and place it adjacent to m-file and hdf5 folders
filE_NAME_csv = 'Ch2_HE53' + 'bb_ph' + bb_ph + 'bb_min' + bb_min + '.csv' # csv file name
# List column headers (below...)
HE53_header = ['m-file','Index','Chlorophyll (ug/L)','CDOM (/m)', 'TSM (g/m^3)','kd320','kd780',
               'Kd412','Kd667','aph443','ag443','amin443','bb_ph667','bb_min667'] 
with open(csv_file_pthwy+filE_NAME_csv, 'w', newline='') as csvfile: # Write new csv file
    # Format csv file (below)
    spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL) #gives a file identifier to the csv file
    spamwriter.writerow(HE53_header) # Add column headers with units
    for i, linE in enumerate(csv_DATA_MATRIX): 
        # For-loop takes rows of data and writes them into csv file
        linE = list(linE) # Take a slice of the numpy matrix and make it into a list
        linE.insert(0,mFILE_LIST[i]) # Insert m-file name at the top of the list
        spamwriter.writerow(linE) # Write the list into csv file
    
 